### **Definir el directorio de trabajo con Google Colab**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.

In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/ICBF/data


In [ ]:
# Librerias relevantes 
import pandas                  as pd
import numpy                   as np
import json
import time
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.api         as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multitest as smm
import os
from scipy import stats, special
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error, 
                             confusion_matrix, 
                             plot_confusion_matrix,
                             f1_score, 
                             precision_score, 
                             recall_score)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree
from sklearn import metrics
from sklearn import neighbors
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
from matplotlib import pyplot
import random
%matplotlib inline
plt.style.use('ggplot')
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
import joblib 
# Opciones de visualización
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **Abrir base de datos unidos**
Abre el archivo `Sociodemo_pre.parquet` (5945349 registros, 148 MB) como un dataframe de nombre `soc`, con toda la base de datos, preprocesada para asignarle a cada variable el tipo de dato correspondiente.

Abre también el diccionario del archivo `sociodemo_datatypes_low_memo.json` que permite asignar a las variables tipos de datos para un uso más eficiente de la memoria (641 MB).

In [ ]:


def print_score(label, prediction, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(label, prediction, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Classification Report:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, prediction)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(label, prediction, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Classification Report:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n") 

In [ ]:
time0 = time.time()
# Abre el diccionario con la estructura de datos definida
his = list()
# Abre el dataframe
for i in range(3):
  tom = pd.read_parquet(f'historical_data/historia_{i + 1}_train.parquet')
  cols_edad = [col for col in tom.columns if col[:4] == 'Edad']
  cols_medidas = [col for col in tom.columns if col[:4] == 'Peso' or col[:5] == 'Talla']
  cols_zscore = [col for col in tom.columns if col[:6] == 'ZScore']
  cols_estado = [col for col in tom.columns if col[:6] == 'Estado']
  cols_tom = ['IdBeneficiario', 'desnutricion_previa', 'tiempo', 'Desnutricion'] + \
    cols_edad + cols_medidas + cols_zscore
  his.append(tom[cols_tom])
#his[2].info()

soc = pd.read_parquet(f'clean_data/sociodemo_models.parquet')
soc = soc.drop(columns=['cod_mpio', 'Id'])


In [ ]:
his[1].columns

Index(['IdBeneficiario', 'desnutricion_previa', 'tiempo', 'Desnutricion',
       'EdadMeses-2', 'EdadMeses-1', 'EdadMeses-0', 'Peso-2', 'Talla-2',
       'Peso-1', 'Talla-1', 'ZScoreTallaEdad-2', 'ZScorePesoEdad-2',
       'ZScorePesoTalla-2', 'ZScoreIMC-2', 'ZScoreTallaEdad-1',
       'ZScorePesoEdad-1', 'ZScorePesoTalla-1', 'ZScoreIMC-1'],
      dtype='object', name='varname')

In [ ]:
modelos = pd.DataFrame([[tiempo, tomas, sociodemo] for tiempo in range(1, 5) 
  for tomas in range(1, 4)
  for sociodemo in range(4)], 
  columns=['tiempo', 'tomas', 'sociodemo'])
modelos[20:30]

,tiempo,tomas,sociodemo
20,2,3,0
21,2,3,1
22,2,3,2
23,2,3,3
24,3,1,0
25,3,1,1
26,3,1,2
27,3,1,3
28,3,2,0
29,3,2,1


In [ ]:
cols_dpto = [col for col in soc.columns if col[:8] == 'cod_dpto']
cols_top = ['Ind_estudia',
            'Edad_padres_menor',
            'ingresos_promP_imp',
            'Edad_padres_mayor',
            'Tip_cuidado_ninos_2',
            'gasto_ppers_imp',
            'porc_gasto_alim',
            'n_privaciones',
            'Ind_nivel_sisben_4',
            'gasto_alim_ppers_imp',
            'Tip_cuidado_ninos_1',
            'n_personas_hogar',
            'Ind_ninguna_discapac',
            'Nivel_educ_madre',
            'Nivel_educ_padre',
            'n_afec_evento_natural',
            'Uni_dias_agua',
            'n_ninos']

In [ ]:
soc_df = list()
# Todas las variables
soc_df.append(soc)
# Excluyendo la información geográfica
soc = soc.drop(columns=cols_dpto)
soc_df.append(soc)
# Tomando solo las variables importantes
soc = soc[['IdBeneficiario'] + cols_top]
soc_df.append(soc)

In [ ]:
parameters = {'max_depth': [2 * i for i in range(1, 10)], 
              'min_samples_leaf':range(1, 5),
              'n_estimators':[50, 100, 150]}
def train_test_rf(df, seed=1, test_size=0.3, parameters=parameters, verbose=1):
  df = df.dropna()
  idB = list(df['IdBeneficiario'].unique())
  train_ids, test_ids = train_test_split(idB, test_size = test_size, random_state=seed)
  df_train_unbalanced = df[df['IdBeneficiario'].isin(train_ids)]
  df_train_positive = df_train_unbalanced[df_train_unbalanced['Desnutricion'] == 1]
  df_train_negative = df_train_unbalanced[df_train_unbalanced['Desnutricion'] == 0]
  df_train_negative_balanced = df_train_negative.sample(n = len(df_train_positive), random_state=seed)
  df_train = pd.concat([df_train_positive, df_train_negative_balanced]).sample(frac=1, random_state=seed)
  df_test = df[df['IdBeneficiario'].isin(test_ids)]
  X_train = df_train.drop(columns=['Desnutricion', 'IdBeneficiario'])
  X_test = df_test.drop(columns=['Desnutricion', 'IdBeneficiario'])
  y_train = df_train['Desnutricion']
  y_test = df_test['Desnutricion']
 
  classifier = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=seed), 
                              parameters, n_jobs=10, cv=5, verbose=verbose) #5 kfolds
  classifier.fit(X=X_train, y=y_train)
  model = classifier.best_estimator_
  y_pred_train = model.predict(X_train)
  y_pred_test = model.predict(X_test)
 
  train_accuracy = accuracy_score(y_train, y_pred_train)
  precision_train = precision_score(y_train, y_pred_train)
  recall_train = recall_score(y_train, y_pred_train)
 
  test_accuracy = accuracy_score(y_test, y_pred_test)
  precision_test = precision_score(y_test, y_pred_test)
  recall_test = recall_score(y_test, y_pred_test)
 
  importance = pd.Series({X_train.columns[i]: model.feature_importances_[i] 
                          for i in range(len(model.feature_importances_))})
  metrics = pd.DataFrame({
      'random_state': seed,
      'accuracy_train': accuracy_score(y_train, y_pred_train),
      'precision_train': precision_score(y_train, y_pred_train),
      'recall_train': recall_score(y_train, y_pred_train),
      'accuracy_test': accuracy_score(y_test, y_pred_test),
      'precision_test': precision_score(y_test, y_pred_test),
      'recall_test': recall_score(y_test, y_pred_test),
      'max_depth': classifier.best_params_['max_depth'],
      'n_trees': classifier.best_params_['n_estimators']
      }, index=[seed])
  return (model, metrics, importance.sort_values(ascending=False))

In [ ]:
metrics_list = list()
importance_list = list()

In [ ]:
t0 = time.time()
for i, model in modelos[11:].iterrows():
  df = his[model['tomas'] - 1]
  df = df[df['tiempo'] == model['tiempo']]
  if model['sociodemo'] > 0:
    df = df.merge(soc_df[model['sociodemo'] - 1], on='IdBeneficiario')
  training = train_test_rf(df=df, seed=i, test_size=0.3, parameters=parameters, verbose=0)
  features = pd.DataFrame({'tiempo': model['tiempo'],
                           'tomas': model['tomas'],
                           'sociodemo': model['sociodemo'],
                           'registros': df.shape[0],
                           'variables': df.shape[1]
                           }, index = [i])
  print(f"\nModelo: {df.shape[1]} variables, incluyendo {model['tomas']} tomas anteriores, " + 
        f"para  {model['tiempo']} meses. Tiempo: {time.time() - t0:.2f} s.")
  model_details = pd.concat([training[1], features], axis=1)
  display(model_details)
  joblib.dump(training[0], f'models/random_forest_history/model_{i}.pickle')
  metrics_list.append(model_details)
  importance_list.append(training[2])
  if i % 5 == 0:
    pd.concat(metrics_list).to_parquet('models/random_forest_history_metrics.parquet')
    pd.concat(importance_list, axis=1).transpose().\
      to_parquet('models/random_forest_history_importance.parquet')
 
metrics = pd.concat(metrics_list)
importance = pd.concat(importance_list, axis=1).transpose()


Modelo: 44 variables, incluyendo 3 tomas anteriores, para  1 meses. Tiempo: 110.94 s.


,random_state,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,max_depth,n_trees,tiempo,tomas,sociodemo,registros,variables
11,11,0.985714,0.981132,0.990476,0.854942,0.341538,0.902439,14,100,1,3,3,5626,44



Modelo: 12 variables, incluyendo 1 tomas anteriores, para  2 meses. Tiempo: 426.92 s.


,random_state,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,max_depth,n_trees,tiempo,tomas,sociodemo,registros,variables
12,12,0.808646,0.768343,0.883742,0.714951,0.200333,0.84778,6,150,2,1,0,60898,12



Modelo: 139 variables, incluyendo 1 tomas anteriores, para  2 meses. Tiempo: 645.78 s.


...

,random_state,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,max_depth,n_trees,tiempo,tomas,sociodemo,registros,variables
45,45,0.935366,0.951899,0.917073,0.797998,0.143705,0.647059,10,150,4,3,1,13626,153



Modelo: 120 variables, incluyendo 3 tomas anteriores, para  4 meses. Tiempo: 7572.62 s.


,random_state,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,max_depth,n_trees,tiempo,tomas,sociodemo,registros,variables
46,46,0.861538,0.883152,0.833333,0.78819,0.173362,0.792271,6,150,4,3,2,13626,120



Modelo: 44 variables, incluyendo 3 tomas anteriores, para  4 meses. Tiempo: 7674.47 s.


,random_state,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,max_depth,n_trees,tiempo,tomas,sociodemo,registros,variables
47,47,0.803991,0.831202,0.762911,0.802611,0.151972,0.766082,2,150,4,3,3,13626,44


In [ ]:

metrics = pd.concat(metrics_list)
importance = pd.concat(importance_list, axis=1).transpose()

In [ ]:
metrics.to_parquet('models/random_forest_history_metrics.parquet')
importance.to_parquet('models/random_forest_history_importance.parquet')

In [ ]:
metrics[metrics['tiempo']==1][metrics.columns[4:]]

,accuracy_test,precision_test,recall_test,max_depth,n_trees,tiempo,tomas,sociodemo,registros,variables
0,0.734197,0.279882,0.855639,6,100,1,1,0,20476,12
1,0.748471,0.254980,0.774194,12,100,1,1,1,8301,139
2,0.781208,0.302469,0.841202,12,50,1,1,2,8301,106
3,0.697547,0.238202,0.851406,4,100,1,1,3,8301,30
4,0.824914,0.293587,0.827684,10,50,1,2,0,15862,19
5,0.867517,0.373860,0.788462,10,100,1,2,1,6471,146
6,0.856907,0.299419,0.844262,14,50,1,2,2,6471,113
7,0.828402,0.299287,0.840000,6,50,1,2,3,6471,37
8,0.830464,0.302817,0.848684,16,150,1,3,0,13588,26
9,0.824882,0.273556,0.818182,2,150,1,3,1,5626,153


In [ ]:
importance.mean().sort_values(ascending=False)[:100]

ZScorePesoTalla-1            0.223570
ZScoreIMC-1                  0.184039
ZScorePesoEdad-1             0.091635
ZScorePesoTalla-2            0.083898
ZScoreIMC-2                  0.069314
ZScorePesoEdad-2             0.050367
ZScorePesoTalla-3            0.048083
ZScorePesoEdad-3             0.043870
ZScoreIMC-3                  0.037566
Peso-1                       0.030234
ZScoreTallaEdad-1            0.026507
Talla-1                      0.020613
EdadMeses-0                  0.018367
EdadMeses-1                  0.018024
ZScoreTallaEdad-2            0.017066
EdadMeses-2                  0.014745
EdadMeses-3                  0.014661
ZScoreTallaEdad-3            0.014076
Peso-2                       0.013363
Talla-2                      0.013295
gasto_ppers_imp              0.012918
ingresos_promP_imp           0.012728
Edad_padres_mayor            0.011746
Edad_padres_menor            0.011556
porc_gasto_alim              0.011277
n_privaciones                0.011004
Talla-3     

,accuracy_test,recall_test
tiempo,,
1,0.867517,0.902439
2,0.856420,0.872024
3,0.838324,0.862336
4,0.816376,0.792271


In [ ]:
t0 = time.time()
for i, model in modelos[11:].iterrows():
  df = his[model['tomas'] - 1]
  df = df[df['tiempo'] == model['tiempo']]
  training = train_test_rf(df=df, seed=i, test_size=0.3, parameters=parameters, verbose=0)
  features = pd.DataFrame({'tiempo': model['tiempo'],
                           'tomas': model['tomas'],
                           'sociodemo': model['sociodemo'],
                           'registros': df.shape[0],
                           'variables': df.shape[1]
                           }, index = [i])
  print(f"\nModelo: {df.shape[1]} variables, incluyendo {model['tomas']} tomas anteriores, " + 
        f"para  {model['tiempo']} meses. Tiempo: {time.time() - t0:.2f} s.")
  model_details = pd.concat([training[1], features], axis=1)
  display(model_details)
  joblib.dump(training[0], f'models/random_forest_history/model_{i}.pickle')
  metrics_list.append(model_details)
  importance_list.append(training[2])
  if i % 5 == 0:
    pd.concat(metrics_list).to_parquet('models/random_forest_history_metrics.parquet')
    pd.concat(importance_list, axis=1).transpose().\
      to_parquet('models/random_forest_history_importance.parquet')

In [ ]:
sns.scatterplot(x=mod['tomas'], y=mod['accuracy_test'], hue=mod['sociodemo'])

In [ ]:
cols_medidas = [col for col in df.columns if col[:8] == 'Medicion']#, 'Peso', 'Talla']
cols_zscore = [col for col in df.columns if col[:6] == 'ZScore']
cols_estado = [col for col in df.columns if col[:6] == 'Estado']
cols_zscore_peso = [col for col in df.columns if col[:10] == 'ZScorePeso']
cols_estado_peso = [col for col in df.columns if col[:10] == 'EstadoPeso']
cols_nan = ['Ind_acudio_salud',
            'Ind_fue_atendido_salud',
            'Ind_recibe_comida',
            'Ind_leer_escribir']

for col in cols_estado:
  df[col] = df[col].astype('category').\
    cat.set_categories(tom_cat[col[:-2]], ordered=True)
  df[col] = df[col].cat.codes
df = df.drop(columns=cols_medidas)
df = df[~df.isna()]

In [ ]:
for col in cols_nan:
  df[col] = df[col].fillna(0)

for col in df.columns[:-2]:
  try:
    df[col] = df[col].fillna(df[col].mean())
  except:
    pass
df.info()
df.head()

In [ ]:
df = df.query('FechaNacimiento > 2010')
df = df.drop(columns=['cod_mpio', 'cod_dpto', 'FechaNacimiento', 'Id',
                 'Ind_grupo_sisben_4'])

In [ ]:
df = df.drop(columns=['EstadoPesoTalla-0'])

In [ ]:
parameters = {'max_depth': [2 * i for i in range(1, 10)], 
              'min_samples_leaf':range(1,5),
              'n_estimators':[50, 100, 150]}

In [ ]:
df = df[df['tiempo'] == 3]
idB = list(df['IdBeneficiario'].unique())
df['Desnutricion'] = df['Desnutricion'].astype('int')
df['Desnutricion'].value_counts(normalize=True)

In [ ]:
df.head()

In [ ]:
m = 2012
# Modelo con todas las variables

train_ids, test_ids = train_test_split(idB, test_size = 0.3, random_state=seed)
df_train_unbalanced = df[df['IdBeneficiario'].isin(train_ids)]
df_train_positive = df_train_unbalanced[df_train_unbalanced['Desnutricion'] == 1]
df_train_negative = df_train_unbalanced[df_train_unbalanced['Desnutricion'] == 0]
df_train_negative_balanced = df_train_negative.sample(n = len(df_train_positive), random_state=seed)
df_train = pd.concat([df_train_positive, df_train_negative_balanced]).sample(frac=1, random_state=seed)
df_test = df[df['IdBeneficiario'].isin(test_ids)]
X_train = df_train.drop(columns=['Desnutricion', 'IdBeneficiario'])
X_test = df_test.drop(columns=['Desnutricion'])
y_train = df_train['Desnutricion']
y_test = df_test['Desnutricion']

classifier = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=seed), 
                             parameters, n_jobs=10, cv=5, verbose=1) #5 kfolds
classifier.fit(X=X_train, y=y_train)
model = classifier.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)

test_accuracy = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)

importance = pd.Series({X_train.columns[i]: model.feature_importances_[i] 
                        for i in range(len(model.feature_importances_))})
metrics_all = pd.DataFrame({
    'model': 'all',
    'accuracy_train': accuracy_score(y_train, y_pred_train),
    'precision_train': precision_score(y_train, y_pred_train),
    'recall_train': recall_score(y_train, y_pred_train),
    'accuracy_test': accuracy_score(y_test, y_pred_test),
    'precision_test': precision_score(y_test, y_pred_test),
    'recall_test': recall_score(y_test, y_pred_test),
    'max_depth': classifier.best_params_['max_depth'],
    'n_trees': classifier.best_params_['n_estimators']
    }, index=[m])
print('Most important variables:')
print(importance.sort_values(ascending=False)[:10])
display(metrics_all)

In [ ]:
y_test.value_counts()

In [ ]:
df = pd.read_parquet('historical_data/historia_3_train.parquet')
df.columns

In [ ]:
df = df.drop(columns=[col for col in df.columns if col[:5] in ['Fecha', 'Estad', 'Prese', 'Contr', 'Codig',
                                                               'Medic']]).\
 dropna()
df = df[df['tiempo'] == 3]
df.info()

In [ ]:
m = 2012
# Modelo con todas las variables

train_ids, test_ids = train_test_split(idB, test_size = 0.3, random_state=seed)
df_train_unbalanced = df[df['IdBeneficiario'].isin(train_ids)]
df_train_positive = df_train_unbalanced[df_train_unbalanced['Desnutricion'] == 1]
df_train_negative = df_train_unbalanced[df_train_unbalanced['Desnutricion'] == 0]
df_train_negative_balanced = df_train_negative.sample(n = len(df_train_positive), random_state=seed)
df_train = pd.concat([df_train_positive, df_train_negative_balanced]).sample(frac=1, random_state=seed)
df_test = df[df['IdBeneficiario'].isin(test_ids)]
X_train = df_train.drop(columns=['Desnutricion', 'IdBeneficiario'])
X_test = df_test.drop(columns=['Desnutricion', 'IdBeneficiario'])
y_train = df_train['Desnutricion']
y_test = df_test['Desnutricion']

classifier = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=seed), 
                             parameters, n_jobs=10, cv=5, verbose=1) #5 kfolds
classifier.fit(X=X_train, y=y_train)
model = classifier.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)

test_accuracy = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)

importance = pd.Series({X_train.columns[i]: model.feature_importances_[i] 
                        for i in range(len(model.feature_importances_))})
metrics_all = pd.DataFrame({
    'model': 'all',
    'accuracy_train': accuracy_score(y_train, y_pred_train),
    'precision_train': precision_score(y_train, y_pred_train),
    'recall_train': recall_score(y_train, y_pred_train),
    'accuracy_test': accuracy_score(y_test, y_pred_test),
    'precision_test': precision_score(y_test, y_pred_test),
    'recall_test': recall_score(y_test, y_pred_test),
    'max_depth': classifier.best_params_['max_depth'],
    'n_trees': classifier.best_params_['n_estimators']
    }, index=[m])
print('Most important variables:')
print(importance.sort_values(ascending=False))
display(metrics_all)

In [ ]:
# Submuestreo de la clase minoritaria
df3_Desnutricion = df3.loc[df3['Desnutricion']]
df3_NoDesnutricion = df3.loc[~df3['Desnutricion']].sample(n=df3['Desnutricion'].sum(), random_state=12)
dfXy = pd.concat([df3_Desnutricion, df3_NoDesnutricion]).sample(frac=1, random_state=12)
dfXy['Desnutricion'] = dfXy['Desnutricion'].astype('float')
dfXy.head()

In [ ]:
X = dfXy.drop(columns=['FechaNacimiento', 'Ind_grupo_sisben_4', 'IdBeneficiario', 'EstadoPesoTalla-0', 'CodigoPesoTalla', 'Desnutricion'])
y = dfXy[['Desnutricion']]
print(f"""X shape {X.shape}
y shape {y.shape}""")

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)

print(f"""X_train shape {X_train.shape}
y_train shape {y_train.shape}
X_test shape {X_test.shape}
y_test shape {y_test.shape}""")

In [ ]:
y_train.value_counts()

In [ ]:
parameters = {'max_depth': [2 * i for i in range(1, 10)], 
              'min_samples_leaf':range(1,5), 'n_estimators':[50, 100, 150]}
classifierRF1 = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=12), 
                             parameters, n_jobs=6, cv=5, verbose=1) #5 kfolds
classifierRF1.fit(X=X_train, y=y_train)
clfRF_model1 = classifierRF1.best_estimator_
print('Best classifierRF ', classifierRF1.best_score_, classifierRF1.best_params_)

***Predicción RF All Features  - Unbalanced dataset***

In [ ]:
pred_clf_rf=clfRF_model1.predict(X_test)
pred_clf_RF_test = clfRF_model1.predict(X_test)
pred_clf_RF_train = clfRF_model1.predict(X_train)
plot_confusion_matrix(clfRF_model1, X_test, y_test, cmap='coolwarm');

In [ ]:
best_model_RF = clfRF_model1 
# get importance
importance = pd.Series(best_model_RF.feature_importances_).reset_index().\
  sort_values(by=0, ascending=False)
# summarize feature importance
for i, row in importance[:10].iterrows():
  print(f'Score: {row[0]:.5%}, Feature: {X.columns[row["index"]]}')
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance[0])
pyplot.show()

In [ ]:
target_names = ['No_desnutricion', 'Desnutricion']


print('_'*20,'pred_clf_rf','_'*20)
print(classification_report(y_test, pred_clf_rf, target_names=target_names))
print('_'*60,'\n')

score_result = dict()
print_score(y_train, pred_clf_RF_train, train=True)
print_score(y_test, pred_clf_RF_test, train=False)

score_result['RF'] = {
        'Train': accuracy_score(y_train,pred_clf_RF_train),
        'Test': accuracy_score(y_test, pred_clf_RF_test)}
print(score_result)

###Los niños del train y del test son distintos

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
id_train, id_test = \
    train_test_split(dfXy['IdBeneficiario'], test_size=0.3, shuffle=True, random_state=12)

df_train = dfXy[dfXy['IdBeneficiario'].isin(id_train)].\
  drop(columns=['IdBeneficiario', 'EstadoPesoTalla-0', 'CodigoPesoTalla'])
df_test = dfXy[dfXy['IdBeneficiario'].isin(id_test)].\
  drop(columns=['IdBeneficiario', 'EstadoPesoTalla-0', 'CodigoPesoTalla'])

X_train = df_train.drop(columns=['Desnutricion'])
X_test = df_test.drop(columns=['Desnutricion'])

y_train = df_train[['Desnutricion']]
y_test = df_test[['Desnutricion']]
print(f"""X shape {X.shape}
y shape {y.shape}""")
print(f"""X_train shape {X_train.shape}
y_train shape {y_train.shape}
X_test shape {X_test.shape}
y_test shape {y_test.shape}""")

y_train.value_counts()

In [ ]:
parameters = {'max_depth': [2 * i for i in range(1, 10)], 
              'min_samples_leaf':range(1,5), 'n_estimators':[50, 100, 150]}
classifierRF2 = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=12), 
                             parameters, n_jobs=6, cv=5, verbose=1) #5 kfolds
classifierRF2.fit(X=X_train, y=y_train)
clfRF_model2 = classifierRF1.best_estimator_
print('Best classifierRF ', classifierRF2.best_score_, classifierRF2.best_params_)

In [ ]:
pred_clf_rf=clfRF_model2.predict(X_test)
pred_clf_RF_test = clfRF_model2.predict(X_test)
pred_clf_RF_train = clfRF_model2.predict(X_train)
plot_confusion_matrix(clfRF_model2, X_test, y_test, cmap='coolwarm');

In [ ]:
target_names = ['No_desnutricion', 'Desnutricion']

print('_'*20,'pred_clf_rf','_'*20)
print(classification_report(y_test, pred_clf_rf, target_names=target_names))
print('_'*60,'\n')

score_result = dict()
print_score(y_train, pred_clf_RF_train, train=True)
print_score(y_test, pred_clf_RF_test, train=False)

score_result['RF'] = {
        'Train': accuracy_score(y_train,pred_clf_RF_train),
        'Test': accuracy_score(y_test, pred_clf_RF_test)}
print(score_result)

In [ ]:
best_model_RF = clfRF_model2
# get importance
importance = pd.Series(best_model_RF.feature_importances_).reset_index().\
  sort_values(by=0, ascending=False)
# summarize feature importance
for i, row in importance[:10].iterrows():
  print(f'Score: {row[0]:.5%}, Feature: {X.columns[row["index"]]}')
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance[0])
pyplot.show()

### Modelo minimo (Z-scores PesoTalla e IMC)

In [ ]:
keep

In [ ]:
# train test split
keep = [col for col in dfXy.columns if col[:-2] in ('ZScorePesoTalla', 'ZScoreIMC', 'EdadMeses')]
X3 = X[keep]
X3_train = X_train[keep]
X3_test = X_test[keep]

print(f"""X shape {X3.shape}
y shape {y.shape}""")
print(f"""X_train shape {X3_train.shape}
y_train shape {y_train.shape}
X_test shape {X3_test.shape}
y_test shape {y_test.shape}""")

y_train.value_counts()

In [ ]:
parameters = {'max_depth': [2 * i for i in range(1, 10)], 
              'min_samples_leaf':range(1,5), 'n_estimators':[50, 100, 150]}
classifierRF3 = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=12), 
                             parameters, n_jobs=6, cv=5, verbose=1) #5 kfolds
classifierRF3.fit(X=X3_train, y=y_train)
clfRF_model3 = classifierRF3.best_estimator_
print('Best classifierRF ', classifierRF3.best_score_, classifierRF3.best_params_)

In [ ]:
pred_clf_rf=clfRF_model3.predict(X3_test)
pred_clf_RF_test = clfRF_model3.predict(X3_test)
pred_clf_RF_train = clfRF_model3.predict(X3_train)
plot_confusion_matrix(clfRF_model3, X3_test, y_test, cmap='coolwarm');

In [ ]:
target_names = ['No_desnutricion', 'Desnutricion']

print('_'*20,'pred_clf_rf','_'*20)
print(classification_report(y_test, pred_clf_rf, target_names=target_names))
print('_'*60,'\n')

score_result = dict()
print_score(y_train, pred_clf_RF_train, train=True)
print_score(y_test, pred_clf_RF_test, train=False)

score_result['RF'] = {
        'Train': accuracy_score(y_train,pred_clf_RF_train),
        'Test': accuracy_score(y_test, pred_clf_RF_test)}
print(score_result)

In [ ]:
best_model_RF = clfRF_model3 
# get importance
importance = pd.Series(best_model_RF.feature_importances_).reset_index().\
  sort_values(by=0, ascending=False)
# summarize feature importance
for i, row in importance[:10].iterrows():
  print(f'Score: {row[0]:.5%}, Feature: {X3.columns[row["index"]]}')
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance[0])
pyplot.show()